In [1]:
from Objects.Transformations import *
from Objects.WSBM import *
from Objects.TWSBMInstance import *

from Computation.Computation import *

In [2]:
N = 4

In [3]:
for rho, pi, model in product(RHOS[:1], PIS[:1], MODELS[:1]):
	print(f"Simulating for rho={rho}, pi={pi}, model={model.__name__}")
	simulate_in_grid(N, model, (rho, pi), transformations=TRANSFORMS)

Simulating for rho=0.25, pi=0.1, model=betaWSBM
Progress: 10/16 (62.5%). Elapsed: 00:00:16. ETA: 00:00:10.


In [4]:
path = "Computation/Grids"
metrics = {}
for rho, pi, model in product(RHOS[:1], PIS[:1], MODELS[:1]):
    grids = np.load(f"{path}/{model.__name__}_{r_dot(rho)}_{r_dot(pi)}.npz")
    metrics[(rho, pi, model)] = {}
    for tid, t in TRANSFORMS_MAP.items():
        metrics[(rho, pi, model)][t] = {}
        for metric in METRICS_ID:
            metrics[(rho, pi, model)][t][metric] = grids[f'{tid}_{metric}']

In [8]:
def stack_grids(m, metric):
    return np.stack([m[t][metric] for t in TRANSFORMS], axis=-1)

def best_transform_metrics(m):
    cgraph = stack_grids(m, 'cgraph')
    cembed = stack_grids(m, 'cembed')
    rand   = stack_grids(m, 'rand')
    max_rand = np.max(rand, axis=-1)

    m['cgraph-Best Transform'] = {}
    BT_arg_cgraph = np.argmax(cgraph, axis=-1)
    m['cgraph-Best Transform']['arg'] = BT_arg_cgraph
    m['cgraph-Best Transform']['rand'] = np.take_along_axis(rand, BT_arg_cgraph[..., None], axis=-1).squeeze(-1)
    m['cgraph-Best Transform']['regret'] = max_rand - m['cgraph-Best Transform']['rand']

    m['cembed-Best Transform'] = {}
    BT_arg_cembed = np.argmax(cembed, axis=-1)
    m['cembed-Best Transform']['arg'] = BT_arg_cembed
    m['cembed-Best Transform']['rand'] = np.take_along_axis(rand, BT_arg_cembed[..., None], axis=-1).squeeze(-1)
    m['cembed-Best Transform']['regret'] = max_rand - m['cembed-Best Transform']['rand']

    return m


In [12]:
from scipy.stats import pearsonr
eps = np.finfo(float).eps

def correlation(m):
    m['Correlation'] = {}
    m['Correlation']['rand'] = {metric: pearsonr(m['rand'].ravel(), m[metric].ravel()) for metric in ['ctrue', 'cgraph', 'cembed']}
    m['Correlation']['ctrue'] = {metric: pearsonr(m['ctrue'].ravel(), m[metric].ravel()) for metric in ['cgraph', 'cembed']}

    return m

def abs_bias(true, pred):
    return np.abs(true - pred)

def rel_bias(true, pred):
    return abs_bias(true, pred) / (true + eps)

def log_bias(true, pred):
    return np.log(pred / (true + eps))

def bias(m):
    m['Bias'] = {}
    m['Bias']['cgraph'] = {'abs': abs_bias(m['ctrue'], m['cgraph']), 'rel': rel_bias(m['ctrue'], m['cgraph']), 'log': log_bias(m['ctrue'], m['cgraph'])}
    m['Bias']['cembed'] = {'abs': abs_bias(m['ctrue'], m['cembed']), 'rel': rel_bias(m['ctrue'], m['cembed']), 'log': log_bias(m['ctrue'], m['cembed'])}

    return m

In [13]:
for rho, pi, model in product(RHOS[:1], PIS[:1], MODELS[:1]):
    m = metrics[(rho, pi, model)]
    metrics[(rho, pi, model)] = best_transform_metrics(m)
    for t in TRANSFORMS:
        m = metrics[(rho, pi, model)][t]
        metrics[(rho, pi, model)][t] = correlation(m)
        metrics[(rho, pi, model)][t] = bias(m)

In [16]:
print(metrics[(0.25, 0.1, betaWSBM)])

{<Objects.Transformations.IdentityTransform object at 0x000002027E411250>: {'ctrue': array([[1.41748132e-02, 4.83740000e-03, 1.76961090e-03, 1.57497924e-03],
       [1.44443046e-02, 4.53533071e-03, 1.03582064e-03, 5.03925635e-04],
       [1.49349620e-02, 4.68673151e-03, 8.76273985e-04, 9.73637761e-05],
       [1.54489253e-02, 4.99975711e-03, 9.71926350e-04, 0.00000000e+00]]), 'cgraph': array([[0.00069981, 0.00069249, 0.00151964, 0.00193682],
       [0.01442373, 0.0007107 , 0.0008487 , 0.00077149],
       [0.0149161 , 0.00079619, 0.00080446, 0.00078131],
       [0.01538965, 0.0007476 , 0.00078429, 0.00081897]]), 'cembed': array([[9.16172897e-05, 1.02432735e-04, 1.01647886e-04, 7.89334650e-04],
       [3.78520063e-03, 1.39025288e-04, 2.47187328e-04, 2.72156560e-04],
       [5.46169921e-03, 1.52589712e-04, 2.42742875e-04, 2.05563934e-04],
       [6.55539328e-03, 1.66982579e-04, 2.61786824e-04, 3.76248485e-04]]), 'rand': array([[ 4.13080537e-02,  7.67583367e-03,  5.20937872e-01,
         8